In [2]:
from src.model.gemma3 import Gemma3Processor, Gemma3ForConditionalGeneration
from PIL import Image
import torch

model_path = "/inspire/ssd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/public/yjc/cunxin/pi0_suite/checkpoints/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_path, device_map="auto"
).eval()

processor = Gemma3Processor.from_pretrained(model_path)

/inspire/ssd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/public/yjc/conda/envs/pi0_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import src.model.gemma3.processing_gemma3 because of the following error (look up to see its traceback):
No module named 'src.feature_extraction_utils'

In [ ]:

eggplant = Image.open("/inspire/ssd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/public/yjc/cunxin/pi0_suite/open-pi-zero/eggplant__conf0.61.jpg")
basket = Image.opne("/inspire/ssd/ws-f4d69b29-e0a5-44e6-bd92-acf4de9990f0/public-project/public/yjc/cunxin/pi0_suite/open-pi-zero/yellow_basket__conf0.17.jpg")

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]},
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What can you see in this image "},
            {"type": "image", "content": eggplant},
            {"type": "text", "text": "and I want to know this one too "},
            {"type": "image", "content": basket},
        ]
    },
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)
